In [48]:
import os

# Set your working directory
os.chdir("D:/work/Trade Analysis")   # <-- your folder path

In [41]:
import os
import pandas as pd
import numpy as np
import quantstats as qs


In [42]:
csv_path = "Master_Data till 31 0ct 2025.csv"

In [43]:
print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close"
    }
)

Loading data...


In [45]:
raw.head()

,Date,Ticker,Open,Close
0,2016-01-04,ACN,101.38,101.86
1,2016-01-05,ACN,102.32,102.38
2,2016-01-06,ACN,102.09,102.18
3,2016-01-07,ACN,100.69,99.16
4,2016-01-08,ACN,99.53,98.15


In [54]:
import pandas as pd

# 1) Put the FULL path to your parquet file here
bt_file_path = "D:/work/Trade Analysis/ACN_USD_D1_20_180_10_2_10_10_Short_BT.parquet"

# 2) Read parquet
#    If you have pyarrow installed, this is safest:
df = pd.read_parquet(bt_file_path, engine="pyarrow")

# 3) Print column names
print("\nColumn names in this parquet file:")
for col in df.columns:
    print(col)

ArrowKeyError: A type extension with name pandas.period already defined

In [50]:
!pip install pyarrow
!pip install fastparquet

   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/28.1 MB 648.1 kB/s eta 0:00:44
   ---------------------------------------- 0.1/28.1 MB 1.2 MB/s eta 0:00:24
   ---------------------------------------- 0.2/28.1 MB 1.5 MB/s eta 0:00:19
   ---------------------------------------- 0.3/28.1 MB 1.1 MB/s eta 0:00:25
    --------------------------------------- 0.4/28.1 MB 1.4 MB/s eta 0:00:20
    --------------------------------------- 0.5/28.1 MB 1.6 MB/s eta 0:00:17
   - -------------------------------------- 0.7/28.1 MB 1.9 MB/s eta 0:00:15
   - -------------------------------------- 0.9/28.1 MB 2.2 MB/s eta 0:00:13
   - -------------------------------------- 1.1/28.1 MB 2.3 MB/s eta 0:00:12
   - -------------------------------------- 1.4/28.1 MB 2.6 MB/s eta 0:00:11
   -- ------------------------------------- 1.5/28.1 MB 2.7 MB/s eta 0:00:10
   -- -----


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\avina\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/671.0 kB ? eta -:--:--
   - ------------------------------------- 20.5/671.0 kB 682.7 kB/s eta 0:00:01
   ---- ---------------------------------- 81.9/671.0 kB 919.0 kB/s eta 0:00:01
   ------------ --------------------------- 204.8/671.0 kB 1.6 MB/s eta 0:00:01
   ------------------ --------------------- 317.4/671.0 kB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 481.3/671.0 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  665.6/671.0 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 671.0/671.0 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 21.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/201.0 kB ? eta -:--:-


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\avina\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "Master_Data till 31 0ct 2025.csv"

# Column names in your CSV
date_col   = "date"
ticker_col = "org_symbol"
open_col   = "askopen"
close_col  = "askclose"

# Strategy parameters
lookback_months_list = [3, 6, 12]      # momentum lookbacks in months
k_list               = [5, 10, 20]     # number of longs and shorts
forward_horizons     = [1]             # holding period in months (1 = rebalance monthly / hold 1M)

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# Output files
rebalance_csv_out   = "momentum_rebalances.csv"
returns_csv_out     = "momentum_returns.csv"
summary_csv_out     = "momentum_summary_2019_onwards.csv"
qs_html_out         = "Momentum_QS_best_2019_onwards.html"


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close"
    }
)

# Drop rows with missing essentials
raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close"])

# Sort by date/ticker
raw = raw.sort_values(["Date", "Ticker"])

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

# Align indices so they cover the same date range
all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

# Rebalance dates: month starts within available data
month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:
        prev_target = dt - pd.DateOffset(months=L)

        # previous and current dates on CLOSE
        idx_prev = dates_index_close.get_indexer([prev_target], method="ffill")[0]
        idx_curr = dates_index_close.get_indexer([dt],          method="ffill")[0]
        if idx_prev == -1 or idx_curr == -1:
            continue

        date_prev = dates_index_close[idx_prev]
        date_curr = dates_index_close[idx_curr]

        close_prev = close_wide.loc[date_prev]
        close_curr = close_wide.loc[date_curr]

        # L-month momentum
        mom = (close_curr / close_prev) - 1.0
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])
            shorts = list(ranked.index[-k:])

            # Entry date: first trading date ON or AFTER dt
            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                # Round-trip transaction cost
                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_ret = rtns_net.loc[valid_longs].mean()

                # Short leg – profit when prices fall
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_ret_raw = rtns_net.loc[valid_shorts].mean()
                short_ret     = -short_ret_raw

                strat_ret = long_ret + short_ret

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "date_prev": date_prev,
                    "date_curr": date_curr,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return": float(strat_ret)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    # Parameter-level summary on 2019+ window
    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    # Approximate monthly equivalents (H is in months)
    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    # Annualized return & vol (from monthly)
    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)

    # Annualized Sharpe (rf = 0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"Parameter summary (2019 onwards) saved to: {summary_csv_out}")

    # Filter for configs with enough history
    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()

    # Prefer H = 1
    perf_valid_H1 = perf_valid[perf_valid["horizon_months"] == 1].copy()
    if perf_valid_H1.empty:
        print("No valid configs with H=1 and sufficient observations (2019+); falling back to all horizons.")
        perf_to_use = perf_valid
    else:
        perf_to_use = perf_valid_H1

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        # Pick best by annualized Sharpe
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        print("\n=== Best configuration (by annualized Sharpe, 2019 onwards) ===")
        print(f"Lookback (months): {best_L}")
        print(f"Number of longs/shorts (k): {best_k}")
        print(f"Holding horizon (months): {best_H}")
        print(f"Annualized return (2019+): {best_row['ann_return']:.2%}")
        print(f"Annualized vol (2019+):    {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe (2019+): {best_row['sharpe_ann']:.2f}")
        print(f"Number of rebalances used: {int(best_row['n_obs'])}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED) & GENERATE HTML
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("rebalance_date")
        )

        # We have one strat_return per rebalance. We also have entry_date/exit_date.
        # We'll convert each monthly return into constant daily returns over [entry_date, exit_date].

        # Ensure dates
        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])
        best_returns_eval = best_returns_eval.sort_values("entry_date")

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]

            # Business days between entry and exit (inclusive)
            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            # Constant daily return such that (1 + r_d)^N = 1 + R
            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            # Assign; if overlap, we add (simple approximation)
            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        # Sort daily series
        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"Momentum_L{best_L}_k{best_k}_H{best_H}"

        # Very simple benchmark: flat zero return
        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        # Make sure output folder exists
        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        # This is the QuantStats tear sheet you wanted (strategy vs benchmark)
        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"Momentum Strategy (Best Config, 2019+) – L={best_L}, k={best_k}, H={best_H}",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")

print("\nFinished.")


Loading data...
Building wide price matrices...
Number of candidate rebalance months: 119
Starting backtest...
Backtest loop completed.
Rebalance book saved to: momentum_rebalances.csv
Returns table saved to:  momentum_returns.csv
Parameter summary (2019 onwards) saved to: momentum_summary_2019_onwards.csv

=== Best configuration (by annualized Sharpe, 2019 onwards) ===
Lookback (months): 6
Number of longs/shorts (k): 10
Holding horizon (months): 1
Annualized return (2019+): 30.63%
Annualized vol (2019+):    31.73%
Annualized Sharpe (2019+): 0.97
Number of rebalances used: 66

Generating QuantStats HTML report...


C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\Momentum_QS_best_2019_onwards.html

Finished.


In [12]:


# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "Master_Data till 31 0ct 2025.csv"

# Column names in your CSV
date_col   = "date"
ticker_col = "org_symbol"
open_col   = "askopen"
close_col  = "askclose"

# Strategy parameters
lookback_months_list = [3, 6, 12]      # momentum lookbacks in months
k_list               = [10]            # number of longs and shorts (you can still add [5, 10, 20] if you want)
forward_horizons     = [1]             # holding period in months (1 = rebalance monthly / hold 1M)

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
# Annual financing rate, e.g. 0.03 = 3% per year
financing_rate_annual = 0.03

# We'll apply this as a *linear* cost over the holding period:
# holding_cost = financing_rate_annual * (days_held / 365)
# and subtract it from strategy return for that trade.

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme: overweight top N inside the k basket
# For longs: top positions (best momentum) get w_top_long, others get w_rest_long
top_bucket_size_long   = 5
top_bucket_weight_long = 2.0
rest_bucket_weight_long = 1.0

# For shorts: worst momentum names (first in 'shorts' list) get top weight
top_bucket_size_short   = 5
top_bucket_weight_short = 2.0
rest_bucket_weight_short = 1.0

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# Output files
rebalance_csv_out   = "momentum_rebalances.csv"
returns_csv_out     = "momentum_returns.csv"
summary_csv_out     = "momentum_summary_2019_onwards.csv"
qs_html_out         = "Momentum_QS_best_2019_onwards.html"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    """
    Build a two-bucket weight vector for a *ranked* list of tickers.
    - First 'top_bucket_size' tickers get weight w_top
    - Remaining tickers get weight w_rest
    Then rescale so weights sum to 1.
    """
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        # fallback to equal-weight
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    """
    Return a weight Series indexed by valid_names for the given side ("long" or "short"),
    according to the global config (weighting_mode, top bucket sizes, etc.).
    """
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        # default fallback: equal weight
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close"
    }
)

# Drop rows with missing essentials
raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close"])

# Sort by date/ticker
raw = raw.sort_values(["Date", "Ticker"])

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

# Align indices so they cover the same date range
all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

# Rebalance dates: month starts within available data
month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:
        prev_target = dt - pd.DateOffset(months=L)

        # previous and current dates on CLOSE
        idx_prev = dates_index_close.get_indexer([prev_target], method="ffill")[0]
        idx_curr = dates_index_close.get_indexer([dt],          method="ffill")[0]
        if idx_prev == -1 or idx_curr == -1:
            continue

        date_prev = dates_index_close[idx_prev]
        date_curr = dates_index_close[idx_curr]

        close_prev = close_wide.loc[date_prev]
        close_curr = close_wide.loc[date_curr]

        # L-month momentum
        mom = (close_curr / close_prev) - 1.0
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])      # best momentum
            shorts = list(ranked.index[-k:])     # worst momentum (ascending)

            # Entry date: first trading date ON or AFTER dt
            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                # Round-trip transaction cost (entry+exit)
                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                # align
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg – profit when prices fall
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw   # short profit is negative of underlying return

                # Strategy return *before* financing cost
                strat_ret_gross = long_ret + short_ret

                # Financing / holding cost for this trade
                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                # Subtract financing cost from overall strategy return
                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "date_prev": date_prev,
                    "date_curr": date_curr,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    # Parameter-level summary on 2019+ window (using NET strat_return)
    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    # Approximate monthly equivalents (H is in months)
    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    # Annualized return & vol (from monthly)
    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)

    # Annualized Sharpe (rf = 0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"Parameter summary (2019 onwards) saved to: {summary_csv_out}")

    # Filter for configs with enough history
    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()

    # Prefer H = 1
    perf_valid_H1 = perf_valid[perf["horizon_months"] == 1].copy()
    if perf_valid_H1.empty:
        print("No valid configs with H=1 and sufficient observations (2019+); falling back to all horizons.")
        perf_to_use = perf_valid
    else:
        perf_to_use = perf_valid_H1

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        # Pick best by annualized Sharpe
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        print("\n=== Best configuration (by annualized Sharpe, 2019 onwards) ===")
        print(f"Lookback (months): {best_L}")
        print(f"Number of longs/shorts (k): {best_k}")
        print(f"Holding horizon (months): {best_H}")
        print(f"Annualized return (2019+): {best_row['ann_return']:.2%}")
        print(f"Annualized vol (2019+):    {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe (2019+): {best_row['sharpe_ann']:.2f}")
        print(f"Number of rebalances used: {int(best_row['n_obs'])}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED) & GENERATE HTML
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        # We have one strat_return (net of tx+financing) per trade, with entry/exit
        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]  # already net of tx + financing

            # Business days between entry and exit (inclusive)
            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            # Constant daily return such that (1 + r_d)^N = 1 + R
            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            # If overlapping windows, combine multiplicatively
            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"Momentum_L{best_L}_k{best_k}_H{best_H}"

        # Simple zero-return benchmark
        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        # Ensure output folder exists
        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"Momentum Strategy (Best Config, 2019+, weighted & financed) – L={best_L}, k={best_k}, H={best_H}",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")

print("\nFinished.")


Loading data...
Building wide price matrices...
Number of candidate rebalance months: 119
Starting backtest...
Backtest loop completed.
Rebalance book saved to: momentum_rebalances.csv
Returns table saved to:  momentum_returns.csv
Parameter summary (2019 onwards) saved to: momentum_summary_2019_onwards.csv

=== Best configuration (by annualized Sharpe, 2019 onwards) ===
Lookback (months): 6
Number of longs/shorts (k): 10
Holding horizon (months): 1
Annualized return (2019+): 26.29%
Annualized vol (2019+):    34.11%
Annualized Sharpe (2019+): 0.77
Number of rebalances used: 66

Generating QuantStats HTML report...


C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\Momentum_QS_best_2019_onwards.html

Finished.


In [30]:
Temp = pd.read_csv("master_file.csv")

In [31]:
Temp.head()

,date,symbol,org_symbol,askopen,askhigh,asklow,askclose,bidopen,bidhigh,bidlow,bidclose,volume
0,2016-01-04,AAO/USD,AAON,22.30,22.47,22.110,22.15,22.30,22.47,22.110,22.15,106008
1,2016-01-05,AAO/USD,AAON,22.20,22.23,21.890,22.10,22.20,22.23,21.890,22.10,96325
2,2016-01-06,AAO/USD,AAON,21.87,22.01,21.595,21.68,21.87,22.01,21.595,21.68,63221
3,2016-01-07,AAO/USD,AAON,21.38,21.47,21.105,21.21,21.38,21.47,21.105,21.21,131504
4,2016-01-08,AAO/USD,AAON,20.95,21.08,20.500,20.51,20.95,21.08,20.500,20.51,117810


In [35]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "master_file.csv"

# Column names in your CSV
date_col   = "date"
ticker_col = "org_symbol"
open_col   = "askopen"
close_col  = "askclose"

# Strategy parameters
lookback_months_list = [3, 6, 12, 24]   # momentum lookbacks in months
k_list               = [5, 10, 20, 50]  # number of longs and shorts
forward_horizons     = [1]              # holding period in months (1 = rebalance monthly / hold 1M)

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
# Annual financing rate, e.g. 0.03 = 3% per year
financing_rate_annual = 0.03

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme for LONGS
top_bucket_size_long    = 5
top_bucket_weight_long  = 2.0
rest_bucket_weight_long = 1.0

# Two-bucket scheme for SHORTS (worst names)
top_bucket_size_short    = 5
top_bucket_weight_short  = 2.0
rest_bucket_weight_short = 1.0

# --- Account / exposure config ---
starting_equity   = 1_000_000.0   # starting AUM = 1M
gross_leverage    = 1.0           # gross exposure = equity * gross_leverage (set 2.0 for 1x long + 1x short)
exposure_csv_out  = "momentum_best_daily_equity_exposure.csv"

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# Output files
rebalance_csv_out   = "momentum_rebalances.csv"
returns_csv_out     = "momentum_returns.csv"
summary_csv_out     = "momentum_summary_2019_onwards.csv"
qs_html_out         = "Momentum_QS_best_2019_onwards.html"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    """
    Build a two-bucket weight vector for a *ranked* list of tickers.
    - First 'top_bucket_size' tickers get weight w_top
    - Remaining tickers get weight w_rest
    Then rescale so weights sum to 1.
    """
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    """
    Return a weight Series indexed by valid_names for the given side ("long" or "short"),
    according to the global config (weighting_mode, top bucket sizes, etc.).
    """
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        # default fallback: equal weight
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close"
    }
)

# Drop rows with missing essentials
raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close"])

# Sort by date/ticker
raw = raw.sort_values(["Date", "Ticker"])

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

# --- ensure DatetimeIndex ---
open_wide.index = pd.to_datetime(open_wide.index)
close_wide.index = pd.to_datetime(close_wide.index)

# Align indices so they cover the same date range
all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

# Rebalance dates: month starts within available data
month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:
        prev_target = dt - pd.DateOffset(months=L)

        # previous and current dates on CLOSE
        idx_prev = dates_index_close.get_indexer([prev_target], method="ffill")[0]
        idx_curr = dates_index_close.get_indexer([dt],          method="ffill")[0]
        if idx_prev == -1 or idx_curr == -1:
            continue

        date_prev = dates_index_close[idx_prev]
        date_curr = dates_index_close[idx_curr]

        close_prev = close_wide.loc[date_prev]
        close_curr = close_wide.loc[date_curr]

        # L-month momentum
        mom = (close_curr / close_prev) - 1.0
        mom = mom.dropna()

        # need enough names to run all k's
        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])      # best momentum
            shorts = list(ranked.index[-k:])     # worst momentum (ascending)

            # Entry date: first trading date ON or AFTER dt
            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                # Round-trip transaction cost (entry+exit)
                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg – profit when prices fall
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw   # short profit is negative of underlying return

                # Strategy return *before* financing cost
                strat_ret_gross = long_ret + short_ret

                # Financing / holding cost for this trade
                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                # Subtract financing cost from overall strategy return
                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "date_prev": date_prev,
                    "date_curr": date_curr,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    # simple sanity check: how many combos actually have trades in 2019+?
    combos_2019 = (
        ret_eval.groupby(["lookback_months", "k", "horizon_months"])
        .size()
        .reset_index(name="n_trades")
    )
    print("\nParameter combinations with trades in 2019+ window:")
    print(combos_2019.to_string(index=False))

    # Parameter-level summary on 2019+ window (using NET strat_return)
    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    # Approximate monthly equivalents (H is in months)
    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    # Annualized return & vol (from monthly)
    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)

    # Annualized Sharpe (rf = 0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"\nParameter summary (2019 onwards) saved to: {summary_csv_out}")

    # Filter for configs with enough history
    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()

    # Prefer H = 1
    perf_valid_H1 = perf_valid[perf["horizon_months"] == 1].copy()
    if perf_valid_H1.empty:
        print("No valid configs with H=1 and sufficient observations (2019+); falling back to all horizons.")
        perf_to_use = perf_valid
    else:
        perf_to_use = perf_valid_H1

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        # Pick best by annualized Sharpe
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        print("\n=== Best configuration (by annualized Sharpe, 2019 onwards) ===")
        print(f"Lookback (months): {best_L}")
        print(f"Number of longs/shorts (k): {best_k}")
        print(f"Holding horizon (months): {best_H}")
        print(f"Annualized return (2019+): {best_row['ann_return']:.2%}")
        print(f"Annualized vol (2019+):    {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe (2019+): {best_row['sharpe_ann']:.2f}")
        print(f"Number of rebalances used: {int(best_row['n_obs'])}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED)
        # AND DAILY EQUITY + EXPOSURE
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]  # already net of tx + financing

            # Business days between entry and exit (inclusive)
            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            # Constant daily return such that (1 + r_d)^N = 1 + R
            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            # If overlapping windows, combine multiplicatively
            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"Momentum_L{best_L}_k{best_k}_H{best_H}"

        # --- NEW: reindex to full business-day calendar and fill gaps with 0 return ---
        if not daily_ret.empty:
            full_idx = pd.bdate_range(daily_ret.index.min(), daily_ret.index.max())
            daily_ret = daily_ret.reindex(full_idx).fillna(0.0)

        # --- Build daily equity from 1M starting capital ---
        daily_equity = (1.0 + daily_ret).cumprod() * starting_equity

        # --- Daily gross exposure based on gross_leverage ---
        # exposure = equity * gross_leverage
        daily_gross_exposure = daily_equity * gross_leverage

        # Save equity & exposure to CSV
        exposure_df = pd.DataFrame({
            "equity": daily_equity,
            "gross_exposure": daily_gross_exposure,
            "exposure_pct_of_initial": daily_gross_exposure / starting_equity
        })
        exposure_df.index.name = "date"
        exposure_df.to_csv(exposure_csv_out)

        # Summary exposure stats
        avg_exposure = daily_gross_exposure.mean()
        min_exposure = daily_gross_exposure.min()
        max_exposure = daily_gross_exposure.max()

        print("\n=== Exposure stats for best configuration (using starting_equity and gross_leverage) ===")
        print(f"Starting equity:        {starting_equity:,.0f}")
        print(f"Gross leverage:         {gross_leverage:.2f}x")
        print(f"Avg gross exposure:     {avg_exposure:,.0f} ({avg_exposure/starting_equity:.2%} of initial 1M)")
        print(f"Min gross exposure:     {min_exposure:,.0f} ({min_exposure/starting_equity:.2%} of initial 1M)")
        print(f"Max gross exposure:     {max_exposure:,.0f} ({max_exposure/starting_equity:.2%} of initial 1M)")
        print(f"Daily equity & exposure saved to: {os.path.abspath(exposure_csv_out)}")

        # Simple zero-return benchmark (for QuantStats)
        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        # Ensure output folder exists
        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"Momentum Strategy (Best Config, 2019+, weighted & financed) – L={best_L}, k={best_k}, H={best_H}",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")

print("\nFinished.")


Loading data...
Building wide price matrices...
Number of candidate rebalance months: 116
Starting backtest...
Backtest loop completed.
Rebalance book saved to: momentum_rebalances.csv
Returns table saved to:  momentum_returns.csv

Parameter combinations with trades in 2019+ window:
 lookback_months  k  horizon_months  n_trades
               3  5               1        76
               3 10               1        76
               3 20               1        77
               3 50               1        77
               6  5               1        76
               6 10               1        78
               6 20               1        79
               6 50               1        77
              12  5               1        76
              12 10               1        79
              12 20               1        79
              12 50               1        78
              24  5               1        77
              24 10               1        79
              24 20       

C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\Momentum_QS_best_2019_onwards.html

Finished.


In [36]:
####Experiment 1: Liquidity + Price Filter Applied

In [37]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "master_file.csv"

# Column names in your CSV
date_col    = "date"
ticker_col  = "org_symbol"
open_col    = "askopen"
close_col   = "askclose"
volume_col  = "volume"   # <-- CHANGE THIS if your column is named differently

# Strategy parameters
lookback_months_list = [3, 6, 12, 24]   # momentum lookbacks in months
k_list               = [5, 10, 20, 50]  # number of longs and shorts
forward_horizons     = [1]              # holding period in months (1 = rebalance monthly / hold 1M)

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
# Annual financing rate, e.g. 0.03 = 3% per year
financing_rate_annual = 0.03

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme for LONGS
top_bucket_size_long    = 5
top_bucket_weight_long  = 2.0
rest_bucket_weight_long = 1.0

# Two-bucket scheme for SHORTS (worst names)
top_bucket_size_short    = 5
top_bucket_weight_short  = 2.0
rest_bucket_weight_short = 1.0

# --- Account / exposure config ---
starting_equity   = 1_000_000.0   # starting AUM = 1M
gross_leverage    = 1.0           # gross exposure = equity * gross_leverage (set 2.0 for 1x long + 1x short)
exposure_csv_out  = "momentum_best_daily_equity_exposure.csv"

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# Output files
rebalance_csv_out   = "momentum_rebalances.csv"
returns_csv_out     = "momentum_returns.csv"
summary_csv_out     = "momentum_summary_2019_onwards.csv"
qs_html_out         = "Momentum_QS_best_2019_onwards.html"

# =====================================================
# NEW: UNIVERSE FILTER CONFIG (liquidity / price)
# =====================================================

use_universe_filters = True

# price filter: require median close over full history >= this
min_median_price = 5.0

# liquidity filter: require median daily dollar volume >= this
# (change threshold based on your units; if prices ~100 and vol ~10k, then 500k–1M is reasonable)
min_median_dollar_vol = 500_000.0

# =====================================================
# NEW: MOMENTUM MODE CONFIG
# =====================================================

# "simple" = L-month return (current behaviour)
# "12_1"   = skip most recent month, use months L_back_to_2m (approx)
# "vol_adj"= L-month return divided by L-month volatility
momentum_mode = "simple"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# HELPER: MOMENTUM SCORE
# =====================================================

def compute_momentum_score(
    close_wide: pd.DataFrame,
    date_curr,
    L: int
) -> pd.Series:
    """
    Compute momentum signal as of date_curr for lookback L months,
    according to momentum_mode.
    """
    dates = close_wide.index

    # base anchor: L months back
    prev_target = date_curr - pd.DateOffset(months=L)
    idx_prev = dates.get_indexer([prev_target], method="ffill")[0]
    idx_curr = dates.get_indexer([date_curr], method="ffill")[0]

    if idx_prev == -1 or idx_curr == -1:
        return pd.Series(dtype=float)

    date_prev = dates[idx_prev]
    date_curr_aligned = dates[idx_curr]

    # basic L-month return
    close_prev = close_wide.loc[date_prev]
    close_curr = close_wide.loc[date_curr_aligned]
    simple_mom = (close_curr / close_prev) - 1.0

    if momentum_mode == "simple":
        return simple_mom.dropna()

    # 12-1 style: ignore last 1 month, use L_back_to_2m
    if momentum_mode == "12_1":
        # approximate: go (L+1) months back, and 1 month back.
        prev_target_far  = date_curr - pd.DateOffset(months=L+1)
        prev_target_near = date_curr - pd.DateOffset(months=1)

        idx_far  = dates.get_indexer([prev_target_far],  method="ffill")[0]
        idx_near = dates.get_indexer([prev_target_near], method="ffill")[0]

        if idx_far == -1 or idx_near == -1:
            return pd.Series(dtype=float)

        date_far  = dates[idx_far]
        date_near = dates[idx_near]

        close_far  = close_wide.loc[date_far]
        close_near = close_wide.loc[date_near]

        mom_12_1 = (close_near / close_far) - 1.0
        return mom_12_1.dropna()

    # volatility-adjusted momentum: return / vol
    if momentum_mode == "vol_adj":
        window_start = date_prev
        window_end   = date_curr_aligned

        # daily returns over [start, end]
        mask = (close_wide.index >= window_start) & (close_wide.index <= window_end)
        cl_slice = close_wide.loc[mask]
        daily_ret = cl_slice.pct_change().dropna(how="all")

        vol = daily_ret.std()  # per symbol
        mom = simple_mom

        # avoid divide by zero
        mom_vol_adj = mom / vol.replace(0, np.nan)
        return mom_vol_adj.dropna()

    # fallback
    return simple_mom.dropna()


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col, volume_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close",
        volume_col: "Volume"
    }
)

# Drop rows with missing essentials
raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close", "Volume"])

# Sort by date/ticker
raw = raw.sort_values(["Date", "Ticker"])

# =====================================================
# APPLY UNIVERSE FILTERS (static, using full history)
# =====================================================

if use_universe_filters:
    print("Applying universe filters (median price / median dollar volume)...")

    # Daily dollar volume = Close * Volume
    raw["DollarVol"] = raw["Close"] * raw["Volume"]

    grouped = raw.groupby("Ticker")

    median_price = grouped["Close"].median()
    median_dv    = grouped["DollarVol"].median()

    eligible = median_price.index[
        (median_price >= min_median_price) &
        (median_dv    >= min_median_dollar_vol)
    ]

    print(f"Total symbols before filters: {raw['Ticker'].nunique()}")
    print(f"Symbols passing filters:       {len(eligible)}")

    raw = raw[raw["Ticker"].isin(eligible)].copy()
else:
    print("Universe filters disabled. Using all symbols.")

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

# ensure DatetimeIndex
open_wide.index = pd.to_datetime(open_wide.index)
close_wide.index = pd.to_datetime(close_wide.index)

# Align indices so they cover the same date range
all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

# Rebalance dates: month starts within available data
month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:

        # Compute momentum scores according to mode
        mom = compute_momentum_score(close_wide, dt, L)
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        # Sort descending: best momentum at top
        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])      # best momentum
            shorts = list(ranked.index[-k:])     # worst momentum

            # Entry date: first trading date ON or AFTER dt
            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                # Round-trip transaction cost (entry+exit)
                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg – profit when prices fall
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw   # short profit is negative of underlying return

                # Strategy return *before* financing cost
                strat_ret_gross = long_ret + short_ret

                # Financing / holding cost for this trade
                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                # Subtract financing cost from overall strategy return
                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    # simple sanity check: how many combos actually have trades in 2019+?
    combos_2019 = (
        ret_eval.groupby(["lookback_months", "k", "horizon_months"])
        .size()
        .reset_index(name="n_trades")
    )
    print("\nParameter combinations with trades in 2019+ window:")
    print(combos_2019.to_string(index=False))

    # Parameter-level summary on 2019+ window (using NET strat_return)
    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    # Approximate monthly equivalents (H is in months)
    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    # Annualized return & vol (from monthly)
    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)

    # Annualized Sharpe (rf = 0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"\nParameter summary (2019 onwards) saved to: {summary_csv_out}")

    # Filter for configs with enough history
    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()

    # Prefer H = 1
    perf_valid_H1 = perf_valid[perf_valid["horizon_months"] == 1].copy()
    if perf_valid_H1.empty:
        print("No valid configs with H=1 and sufficient observations (2019+); falling back to all horizons.")
        perf_to_use = perf_valid
    else:
        perf_to_use = perf_valid_H1

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        # Pick best by annualized Sharpe
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        print("\n=== Best configuration (by annualized Sharpe, 2019 onwards) ===")
        print(f"Lookback (months): {best_L}")
        print(f"Number of longs/shorts (k): {best_k}")
        print(f"Holding horizon (months): {best_H}")
        print(f"Annualized return (2019+): {best_row['ann_return']:.2%}")
        print(f"Annualized vol (2019+):    {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe (2019+): {best_row['sharpe_ann']:.2f}")
        print(f"Number of rebalances used: {int(best_row['n_obs'])}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED)
        # AND DAILY EQUITY + EXPOSURE
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]  # already net of tx + financing

            # Business days between entry and exit (inclusive)
            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            # Constant daily return such that (1 + r_d)^N = 1 + R
            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            # If overlapping windows, combine multiplicatively
            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"Momentum_L{best_L}_k{best_k}_H{best_H}"

        # Reindex to full business-day calendar, fill gaps with 0 return
        if not daily_ret.empty:
            full_idx = pd.bdate_range(daily_ret.index.min(), daily_ret.index.max())
            daily_ret = daily_ret.reindex(full_idx).fillna(0.0)

        # --- Build daily equity from 1M starting capital ---
        daily_equity = (1.0 + daily_ret).cumprod() * starting_equity

        # --- Daily gross exposure based on gross_leverage ---
        daily_gross_exposure = daily_equity * gross_leverage

        # Save equity & exposure to CSV
        exposure_df = pd.DataFrame({
            "equity": daily_equity,
            "gross_exposure": daily_gross_exposure,
            "exposure_pct_of_initial": daily_gross_exposure / starting_equity
        })
        exposure_df.index.name = "date"
        exposure_df.to_csv(exposure_csv_out)

        # Simple zero-return benchmark (for QuantStats)
        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        # Ensure output folder exists
        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"Momentum Strategy (Best Config, 2019+, mode={momentum_mode}) – L={best_L}, k={best_k}, H={best_H}",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")

print("\nFinished.")


Loading data...
Applying universe filters (median price / median dollar volume)...
Total symbols before filters: 2030
Symbols passing filters:       1708
Building wide price matrices...
Number of candidate rebalance months: 116
Starting backtest...
Backtest loop completed.
Rebalance book saved to: momentum_rebalances.csv
Returns table saved to:  momentum_returns.csv

Parameter combinations with trades in 2019+ window:
 lookback_months  k  horizon_months  n_trades
               3  5               1        76
               3 10               1        76
               3 20               1        77
               3 50               1        75
               6  5               1        76
               6 10               1        79
               6 20               1        79
               6 50               1        76
              12  5               1        77
              12 10               1        79
              12 20               1        79
              12 50       

C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\Momentum_QS_best_2019_onwards.html

Finished.


In [38]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# EXPERIMENT TAG (UNIQUE OUTPUT FILENAMES)
# =====================================================

exp_tag = "exp2_liquidity_12_1_momentum"  # <-- change per experiment

print(f"\n===== RUNNING {exp_tag} =====\n")

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "master_file.csv"

# Column names in your CSV
date_col    = "date"
ticker_col  = "org_symbol"
open_col    = "askopen"
close_col   = "askclose"
volume_col  = "volume"   # <-- change if your volume column has a different name

# Strategy parameters
lookback_months_list = [3, 6, 12, 24]   # momentum lookbacks in months
k_list               = [5, 10, 20, 50]  # number of longs and shorts
forward_horizons     = [1]              # holding period in months (1 = rebalance monthly / hold 1M)

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
# Annual financing rate, e.g. 0.03 = 3% per year
financing_rate_annual = 0.03

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme for LONGS
top_bucket_size_long    = 5
top_bucket_weight_long  = 2.0
rest_bucket_weight_long = 1.0

# Two-bucket scheme for SHORTS (worst names)
top_bucket_size_short    = 5
top_bucket_weight_short  = 2.0
rest_bucket_weight_short = 1.0

# --- Account / exposure config ---
starting_equity   = 1_000_000.0   # starting AUM = 1M
gross_leverage    = 1.0           # gross exposure = equity * gross_leverage (set 2.0 for 1x long + 1x short)

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# =====================================================
# UNIVERSE FILTER CONFIG (liquidity / price)
# =====================================================

use_universe_filters = True

# price filter: require median close over full history >= this
min_median_price = 5.0

# liquidity filter: require median daily dollar volume >= this
min_median_dollar_vol = 500_000.0

# =====================================================
# MOMENTUM MODE CONFIG (THIS EXPERIMENT USES 12-1)
# =====================================================

# "simple" = L-month return
# "12_1"   = skip most recent month, use approx months L+1 to 1 month ago
# "vol_adj"= L-month return divided by L-month volatility
momentum_mode = "12_1"

# =====================================================
# OUTPUT FILENAMES (UNIQUE PER EXPERIMENT)
# =====================================================

rebalance_csv_out   = f"{exp_tag}_rebalances.csv"
returns_csv_out     = f"{exp_tag}_returns.csv"
summary_csv_out     = f"{exp_tag}_summary_2019_onwards.csv"
qs_html_out         = f"{exp_tag}_QS_best_2019_onwards.html"
exposure_csv_out    = f"{exp_tag}_daily_equity_exposure.csv"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# HELPER: MOMENTUM SCORE (supports simple / 12_1 / vol_adj)
# =====================================================

def compute_momentum_score(
    close_wide: pd.DataFrame,
    date_curr,
    L: int
) -> pd.Series:
    dates = close_wide.index

    # base anchor: L months back
    prev_target = date_curr - pd.DateOffset(months=L)
    idx_prev = dates.get_indexer([prev_target], method="ffill")[0]
    idx_curr = dates.get_indexer([date_curr], method="ffill")[0]

    if idx_prev == -1 or idx_curr == -1:
        return pd.Series(dtype=float)

    date_prev = dates[idx_prev]
    date_curr_aligned = dates[idx_curr]

    # basic L-month return
    close_prev = close_wide.loc[date_prev]
    close_curr = close_wide.loc[date_curr_aligned]
    simple_mom = (close_curr / close_prev) - 1.0

    if momentum_mode == "simple":
        return simple_mom.dropna()

    # 12-1 style: ignore last 1 month, use approx [L+1 months ago → 1 month ago]
    if momentum_mode == "12_1":
        prev_target_far  = date_curr - pd.DateOffset(months=L+1)
        prev_target_near = date_curr - pd.DateOffset(months=1)

        idx_far  = dates.get_indexer([prev_target_far],  method="ffill")[0]
        idx_near = dates.get_indexer([prev_target_near], method="ffill")[0]

        if idx_far == -1 or idx_near == -1:
            return pd.Series(dtype=float)

        date_far  = dates[idx_far]
        date_near = dates[idx_near]

        close_far  = close_wide.loc[date_far]
        close_near = close_wide.loc[date_near]

        mom_12_1 = (close_near / close_far) - 1.0
        return mom_12_1.dropna()

    # volatility-adjusted momentum: return / vol
    if momentum_mode == "vol_adj":
        window_start = date_prev
        window_end   = date_curr_aligned

        mask = (close_wide.index >= window_start) & (close_wide.index <= window_end)
        cl_slice = close_wide.loc[mask]
        daily_ret = cl_slice.pct_change().dropna(how="all")

        vol = daily_ret.std()  # per symbol
        mom = simple_mom

        mom_vol_adj = mom / vol.replace(0, np.nan)
        return mom_vol_adj.dropna()

    return simple_mom.dropna()


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

# Keep only needed columns and standardize names
raw = raw[[date_col, ticker_col, open_col, close_col, volume_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close",
        volume_col: "Volume"
    }
)

# Drop rows with missing essentials
raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close", "Volume"])

# Sort by date/ticker
raw = raw.sort_values(["Date", "Ticker"])

# =====================================================
# APPLY UNIVERSE FILTERS (static, using full history)
# =====================================================

if use_universe_filters:
    print("Applying universe filters (median price / median dollar volume)...")

    raw["DollarVol"] = raw["Close"] * raw["Volume"]

    grouped = raw.groupby("Ticker")
    median_price = grouped["Close"].median()
    median_dv    = grouped["DollarVol"].median()

    eligible = median_price.index[
        (median_price >= min_median_price) &
        (median_dv    >= min_median_dollar_vol)
    ]

    print(f"Total symbols before filters: {raw['Ticker'].nunique()}")
    print(f"Symbols passing filters:       {len(eligible)}")

    raw = raw[raw["Ticker"].isin(eligible)].copy()
else:
    print("Universe filters disabled. Using all symbols.")

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

# ensure DatetimeIndex
open_wide.index = pd.to_datetime(open_wide.index)
close_wide.index = pd.to_datetime(close_wide.index)

# Align indices so they cover the same date range
all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

# Rebalance dates: month starts within available data
month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:

        mom = compute_momentum_score(close_wide, dt, L)
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])
            shorts = list(ranked.index[-k:])

            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                # Round-trip transaction cost (entry+exit)
                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg – profit when prices fall
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw

                strat_ret_gross = long_ret + short_ret

                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    combos_2019 = (
        ret_eval.groupby(["lookback_months", "k", "horizon_months"])
        .size()
        .reset_index(name="n_trades")
    )
    print("\nParameter combinations with trades in 2019+ window:")
    print(combos_2019.to_string(index=False))

    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)

    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"\nParameter summary (2019 onwards) saved to: {summary_csv_out}")

    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()

    perf_valid_H1 = perf_valid[perf_valid["horizon_months"] == 1].copy()
    if perf_valid_H1.empty:
        print("No valid configs with H=1 and sufficient observations (2019+); falling back to all horizons.")
        perf_to_use = perf_valid
    else:
        perf_to_use = perf_valid_H1

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        best_sharpe = best_row["sharpe_ann"]
        best_annret = best_row["ann_return"]

        # Simple quality label so you see it immediately
        if best_sharpe > 1.0:
            quality = "✅ GOOD (Sharpe > 1.0)"
        elif best_sharpe > 0.0:
            quality = "🟡 OK (Sharpe > 0)"
        else:
            quality = "❌ BAD (Sharpe ≤ 0)"

        print("\n=== BEST CONFIGURATION SUMMARY (2019 onwards) ===")
        print(f"Experiment:             {exp_tag}")
        print(f"Momentum mode:          {momentum_mode}")
        print(f"Lookback (months):      {best_L}")
        print(f"Number of longs/shorts: {best_k}")
        print(f"Holding horizon (M):    {best_H}")
        print(f"Annualized return:      {best_annret:.2%}")
        print(f"Annualized vol:         {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe:      {best_sharpe:.2f}")
        print(f"Number of rebalances:   {int(best_row['n_obs'])}")
        print(f"Iteration quality:      {quality}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED)
        # AND DAILY EQUITY + EXPOSURE
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]

            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"{exp_tag}_L{best_L}_k{best_k}_H{best_H}"

        if not daily_ret.empty:
            full_idx = pd.bdate_range(daily_ret.index.min(), daily_ret.index.max())
            daily_ret = daily_ret.reindex(full_idx).fillna(0.0)

        daily_equity = (1.0 + daily_ret).cumprod() * starting_equity
        daily_gross_exposure = daily_equity * gross_leverage

        exposure_df = pd.DataFrame({
            "equity": daily_equity,
            "gross_exposure": daily_gross_exposure,
            "exposure_pct_of_initial": daily_gross_exposure / starting_equity
        })
        exposure_df.index.name = "date"
        exposure_df.to_csv(exposure_csv_out)

        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"{exp_tag} – Best Config (mode={momentum_mode})",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")
        print(f"Daily equity & exposure saved to: {os.path.abspath(exposure_csv_out)}")

print("\nFinished.")



===== RUNNING exp2_liquidity_12_1_momentum =====

Loading data...
Applying universe filters (median price / median dollar volume)...
Total symbols before filters: 2030
Symbols passing filters:       1708
Building wide price matrices...
Number of candidate rebalance months: 116
Starting backtest...
Backtest loop completed.
Rebalance book saved to: exp2_liquidity_12_1_momentum_rebalances.csv
Returns table saved to:  exp2_liquidity_12_1_momentum_returns.csv

Parameter combinations with trades in 2019+ window:
 lookback_months  k  horizon_months  n_trades
               3  5               1        76
               3 10               1        76
               3 20               1        76
               3 50               1        76
               6  5               1        75
               6 10               1        77
               6 20               1        78
               6 50               1        76
              12  5               1        75
              12 10        

C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\exp2_liquidity_12_1_momentum_QS_best_2019_onwards.html
Daily equity & exposure saved to: D:\work\Trade Analysis\exp2_liquidity_12_1_momentum_daily_equity_exposure.csv

Finished.


In [ ]:
Experiment 3

In [39]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# EXPERIMENT TAG (UNIQUE OUTPUT FILENAMES)
# =====================================================

exp_tag = "exp3_liquidity_vol_adj_momentum"  # <-- change per experiment

print(f"\n===== RUNNING {exp_tag} =====\n")

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "master_file.csv"

# Column names in your CSV
date_col    = "date"
ticker_col  = "org_symbol"
open_col    = "askopen"
close_col   = "askclose"
volume_col  = "volume"   # <-- change if your volume column has a different name

# Strategy parameters
lookback_months_list = [3, 6, 12, 24]   # momentum lookbacks in months
k_list               = [5, 10, 20, 50]  # number of longs and shorts
forward_horizons     = [1]              # holding period in months

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
financing_rate_annual = 0.03  # 3% per year

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme for LONGS
top_bucket_size_long    = 5
top_bucket_weight_long  = 2.0
rest_bucket_weight_long = 1.0

# Two-bucket scheme for SHORTS (worst names)
top_bucket_size_short    = 5
top_bucket_weight_short  = 2.0
rest_bucket_weight_short = 1.0

# --- Account / exposure config ---
starting_equity   = 1_000_000.0   # starting AUM = 1M
gross_leverage    = 1.0           # set 2.0 for 1x long + 1x short

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# =====================================================
# UNIVERSE FILTER CONFIG (liquidity / price)
# =====================================================

use_universe_filters = True

min_median_price       = 5.0        # price filter
min_median_dollar_vol  = 500_000.0  # liquidity filter (Close * Volume median)

# =====================================================
# MOMENTUM MODE CONFIG (THIS EXPERIMENT USES VOL-ADJUSTED)
# =====================================================

# "simple" = L-month return
# "12_1"   = skip recent month
# "vol_adj"= L-month return divided by L-month volatility
momentum_mode = "vol_adj"

# =====================================================
# OUTPUT FILENAMES (UNIQUE PER EXPERIMENT)
# =====================================================

rebalance_csv_out   = f"{exp_tag}_rebalances.csv"
returns_csv_out     = f"{exp_tag}_returns.csv"
summary_csv_out     = f"{exp_tag}_summary_2019_onwards.csv"
qs_html_out         = f"{exp_tag}_QS_best_2019_onwards.html"
exposure_csv_out    = f"{exp_tag}_daily_equity_exposure.csv"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# HELPER: MOMENTUM SCORE (supports simple / 12_1 / vol_adj)
# =====================================================

def compute_momentum_score(
    close_wide: pd.DataFrame,
    date_curr,
    L: int
) -> pd.Series:
    dates = close_wide.index

    # base anchor: L months back
    prev_target = date_curr - pd.DateOffset(months=L)
    idx_prev = dates.get_indexer([prev_target], method="ffill")[0]
    idx_curr = dates.get_indexer([date_curr], method="ffill")[0]

    if idx_prev == -1 or idx_curr == -1:
        return pd.Series(dtype=float)

    date_prev = dates[idx_prev]
    date_curr_aligned = dates[idx_curr]

    close_prev = close_wide.loc[date_prev]
    close_curr = close_wide.loc[date_curr_aligned]
    simple_mom = (close_curr / close_prev) - 1.0

    if momentum_mode == "simple":
        return simple_mom.dropna()

    if momentum_mode == "12_1":
        prev_target_far  = date_curr - pd.DateOffset(months=L+1)
        prev_target_near = date_curr - pd.DateOffset(months=1)

        idx_far  = dates.get_indexer([prev_target_far],  method="ffill")[0]
        idx_near = dates.get_indexer([prev_target_near], method="ffill")[0]

        if idx_far == -1 or idx_near == -1:
            return pd.Series(dtype=float)

        date_far  = dates[idx_far]
        date_near = dates[idx_near]

        close_far  = close_wide.loc[date_far]
        close_near = close_wide.loc[date_near]

        mom_12_1 = (close_near / close_far) - 1.0
        return mom_12_1.dropna()

    if momentum_mode == "vol_adj":
        # use simple L-month return but divide by vol over same window
        window_start = date_prev
        window_end   = date_curr_aligned

        mask = (close_wide.index >= window_start) & (close_wide.index <= window_end)
        cl_slice = close_wide.loc[mask]
        daily_ret = cl_slice.pct_change().dropna(how="all")

        vol = daily_ret.std()  # per symbol
        mom = simple_mom

        mom_vol_adj = mom / vol.replace(0, np.nan)
        return mom_vol_adj.dropna()

    return simple_mom.dropna()


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

raw = raw[[date_col, ticker_col, open_col, close_col, volume_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close",
        volume_col: "Volume"
    }
)

raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close", "Volume"])
raw = raw.sort_values(["Date", "Ticker"])

# =====================================================
# APPLY UNIVERSE FILTERS (static, using full history)
# =====================================================

if use_universe_filters:
    print("Applying universe filters (median price / median dollar volume)...")

    raw["DollarVol"] = raw["Close"] * raw["Volume"]

    grouped = raw.groupby("Ticker")
    median_price = grouped["Close"].median()
    median_dv    = grouped["DollarVol"].median()

    eligible = median_price.index[
        (median_price >= min_median_price) &
        (median_dv    >= min_median_dollar_vol)
    ]

    print(f"Total symbols before filters: {raw['Ticker'].nunique()}")
    print(f"Symbols passing filters:       {len(eligible)}")

    raw = raw[raw["Ticker"].isin(eligible)].copy()
else:
    print("Universe filters disabled. Using all symbols.")

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

open_wide.index = pd.to_datetime(open_wide.index)
close_wide.index = pd.to_datetime(close_wide.index)

all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:

        mom = compute_momentum_score(close_wide, dt, L)
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])
            shorts = list(ranked.index[-k:])

            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw

                strat_ret_gross = long_ret + short_ret

                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    combos_2019 = (
        ret_eval.groupby(["lookback_months", "k", "horizon_months"])
        .size()
        .reset_index(name="n_trades")
    )
    print("\nParameter combinations with trades in 2019+ window:")
    print(combos_2019.to_string(index=False))

    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"\nParameter summary (2019 onwards) saved to: {summary_csv_out}")

    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()
    perf_valid_H1 = perf_valid[perf_valid["horizon_months"] == 1].copy()
    perf_to_use = perf_valid_H1 if not perf_valid_H1.empty else perf_valid

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        best_sharpe = best_row["sharpe_ann"]
        best_annret = best_row["ann_return"]

        if best_sharpe > 1.0:
            quality = "✅ GOOD (Sharpe > 1.0)"
        elif best_sharpe > 0.0:
            quality = "🟡 OK (Sharpe > 0)"
        else:
            quality = "❌ BAD (Sharpe ≤ 0)"

        print("\n=== BEST CONFIGURATION SUMMARY (2019 onwards) ===")
        print(f"Experiment:             {exp_tag}")
        print(f"Momentum mode:          {momentum_mode}")
        print(f"Lookback (months):      {best_L}")
        print(f"Number of longs/shorts: {best_k}")
        print(f"Holding horizon (M):    {best_H}")
        print(f"Annualized return:      {best_annret:.2%}")
        print(f"Annualized vol:         {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe:      {best_sharpe:.2f}")
        print(f"Number of rebalances:   {int(best_row['n_obs'])}")
        print(f"Iteration quality:      {quality}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED)
        # AND DAILY EQUITY + EXPOSURE
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]

            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"{exp_tag}_L{best_L}_k{best_k}_H{best_H}"

        if not daily_ret.empty:
            full_idx = pd.bdate_range(daily_ret.index.min(), daily_ret.index.max())
            daily_ret = daily_ret.reindex(full_idx).fillna(0.0)

        daily_equity = (1.0 + daily_ret).cumprod() * starting_equity
        daily_gross_exposure = daily_equity * gross_leverage

        exposure_df = pd.DataFrame({
            "equity": daily_equity,
            "gross_exposure": daily_gross_exposure,
            "exposure_pct_of_initial": daily_gross_exposure / starting_equity
        })
        exposure_df.index.name = "date"
        exposure_df.to_csv(exposure_csv_out)

        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"{exp_tag} – Best Config (mode={momentum_mode})",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")
        print(f"Daily equity & exposure saved to: {os.path.abspath(exposure_csv_out)}")

print("\nFinished.")



===== RUNNING exp3_liquidity_vol_adj_momentum =====

Loading data...
Applying universe filters (median price / median dollar volume)...
Total symbols before filters: 2030
Symbols passing filters:       1708
Building wide price matrices...
Number of candidate rebalance months: 116
Starting backtest...


C:\Users\avina\AppData\Local\Temp\ipykernel_3136\3974421856.py:203: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = cl_slice.pct_change().dropna(how="all")
C:\Users\avina\AppData\Local\Temp\ipykernel_3136\3974421856.py:203: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = cl_slice.pct_change().dropna(how="all")
C:\Users\avina\AppData\Local\Temp\ipykernel_3136\3974421856.py:203: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or 

Backtest loop completed.
Rebalance book saved to: exp3_liquidity_vol_adj_momentum_rebalances.csv
Returns table saved to:  exp3_liquidity_vol_adj_momentum_returns.csv

Parameter combinations with trades in 2019+ window:
 lookback_months  k  horizon_months  n_trades
               3  5               1        75
               3 10               1        76
               3 20               1        77
               3 50               1        74
               6  5               1        76
               6 10               1        76
               6 20               1        76
               6 50               1        75
              12  5               1        75
              12 10               1        75
              12 20               1        76
              12 50               1        77
              24  5               1        75
              24 10               1        76
              24 20               1        76
              24 50               1        77

C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\exp3_liquidity_vol_adj_momentum_QS_best_2019_onwards.html
Daily equity & exposure saved to: D:\work\Trade Analysis\exp3_liquidity_vol_adj_momentum_daily_equity_exposure.csv

Finished.


In [40]:
import os
import pandas as pd
import numpy as np
import quantstats as qs

# =====================================================
# EXPERIMENT TAG (UNIQUE OUTPUT FILENAMES)
# =====================================================

exp_tag = "exp4_liquidity_vol_adj_12_1_momentum"

print(f"\n===== RUNNING {exp_tag} =====\n")

# =====================================================
# CONFIG
# =====================================================

# Path to your master price data
csv_path = "master_file.csv"

# Column names in your CSV
date_col    = "date"
ticker_col  = "org_symbol"
open_col    = "askopen"
close_col   = "askclose"
volume_col  = "volume"   # <-- change if your volume column has a different name

# Strategy parameters
lookback_months_list = [3, 6, 12, 24]   # momentum lookbacks in months
k_list               = [5, 10, 20, 50]  # number of longs and shorts
forward_horizons     = [1]              # holding period in months

tx_cost_pct = 0.001  # 0.1% per side; total 0.2% per round trip

# --- Financing cost (holding cost) ---
financing_rate_annual = 0.03  # 3% per year

# --- Rank-based weighting config ---
# weighting_mode: "equal" or "two_bucket"
weighting_mode = "two_bucket"

# Two-bucket scheme for LONGS
top_bucket_size_long    = 5
top_bucket_weight_long  = 2.0
rest_bucket_weight_long = 1.0

# Two-bucket scheme for SHORTS (worst names)
top_bucket_size_short    = 5
top_bucket_weight_short  = 2.0
rest_bucket_weight_short = 1.0

# --- Account / exposure config ---
starting_equity   = 1_000_000.0   # starting AUM = 1M
gross_leverage    = 1.0           # set 2.0 for 1x long + 1x short

# Evaluation window (optimize only on this window)
evaluation_start = pd.Timestamp("2019-01-01")

# Minimum number of rebalances to consider a config valid
min_obs_for_config = 24   # e.g. 2 years of monthly data

# =====================================================
# UNIVERSE FILTER CONFIG (liquidity / price)
# =====================================================

use_universe_filters = True

min_median_price       = 5.0        # price filter
min_median_dollar_vol  = 500_000.0  # liquidity filter (Close * Volume median)

# =====================================================
# MOMENTUM MODE CONFIG (THIS EXPERIMENT USES VOL-ADJ 12–1)
# =====================================================

# Options (we'll use "vol_adj_12_1" for this experiment):
# "simple"       = L-month return
# "12_1"         = skip recent 1 month, use [L+1 → 1 month ago]
# "vol_adj"      = L-month return / vol over same window
# "vol_adj_12_1" = 12–1 return / vol over same 12–1 window
momentum_mode = "vol_adj_12_1"

# =====================================================
# OUTPUT FILENAMES (UNIQUE PER EXPERIMENT)
# =====================================================

rebalance_csv_out   = f"{exp_tag}_rebalances.csv"
returns_csv_out     = f"{exp_tag}_returns.csv"
summary_csv_out     = f"{exp_tag}_summary_2019_onwards.csv"
qs_html_out         = f"{exp_tag}_QS_best_2019_onwards.html"
exposure_csv_out    = f"{exp_tag}_daily_equity_exposure.csv"


# =====================================================
# HELPER: RANK-BASED WEIGHTS
# =====================================================

def make_two_bucket_weights(
    tickers_ordered,
    top_bucket_size,
    w_top,
    w_rest
) -> pd.Series:
    n = len(tickers_ordered)
    if n == 0:
        return pd.Series(dtype=float)

    top = min(top_bucket_size, n)
    raw = np.array([w_top] * top + [w_rest] * (n - top), dtype=float)
    raw_sum = raw.sum()
    if raw_sum == 0:
        w = np.repeat(1.0 / n, n)
    else:
        w = raw / raw_sum

    return pd.Series(w, index=tickers_ordered)


def make_weights_for_side(
    valid_names,
    side: str = "long"
) -> pd.Series:
    n = len(valid_names)
    if n == 0:
        return pd.Series(dtype=float)

    if weighting_mode == "equal":
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)

    elif weighting_mode == "two_bucket":
        if side == "long":
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_long,
                w_top=top_bucket_weight_long,
                w_rest=rest_bucket_weight_long
            )
        else:  # short side
            return make_two_bucket_weights(
                tickers_ordered=valid_names,
                top_bucket_size=top_bucket_size_short,
                w_top=top_bucket_weight_short,
                w_rest=rest_bucket_weight_short
            )
    else:
        w = np.repeat(1.0 / n, n)
        return pd.Series(w, index=valid_names)


# =====================================================
# HELPER: MOMENTUM SCORE (supports simple / 12_1 / vol_adj / vol_adj_12_1)
# =====================================================

def compute_momentum_score(
    close_wide: pd.DataFrame,
    date_curr,
    L: int
) -> pd.Series:
    dates = close_wide.index

    # base anchor: L months back
    prev_target = date_curr - pd.DateOffset(months=L)
    idx_prev = dates.get_indexer([prev_target], method="ffill")[0]
    idx_curr = dates.get_indexer([date_curr], method="ffill")[0]

    if idx_prev == -1 or idx_curr == -1:
        return pd.Series(dtype=float)

    date_prev = dates[idx_prev]
    date_curr_aligned = dates[idx_curr]

    close_prev = close_wide.loc[date_prev]
    close_curr = close_wide.loc[date_curr_aligned]
    simple_mom = (close_curr / close_prev) - 1.0

    # --- Simple L-month return ---
    if momentum_mode == "simple":
        return simple_mom.dropna()

    # --- 12–1 momentum: skip most recent 1 month ---
    if momentum_mode == "12_1":
        prev_target_far  = date_curr - pd.DateOffset(months=L+1)
        prev_target_near = date_curr - pd.DateOffset(months=1)

        idx_far  = dates.get_indexer([prev_target_far],  method="ffill")[0]
        idx_near = dates.get_indexer([prev_target_near], method="ffill")[0]

        if idx_far == -1 or idx_near == -1:
            return pd.Series(dtype=float)

        date_far  = dates[idx_far]
        date_near = dates[idx_near]

        close_far  = close_wide.loc[date_far]
        close_near = close_wide.loc[date_near]

        mom_12_1 = (close_near / close_far) - 1.0
        return mom_12_1.dropna()

    # --- Vol-adjusted L-month momentum: return / vol ---
    if momentum_mode == "vol_adj":
        window_start = date_prev
        window_end   = date_curr_aligned

        mask = (close_wide.index >= window_start) & (close_wide.index <= window_end)
        cl_slice = close_wide.loc[mask]
        daily_ret = cl_slice.pct_change().dropna(how="all")

        vol = daily_ret.std()  # per symbol
        mom = simple_mom

        mom_vol_adj = mom / vol.replace(0, np.nan)
        return mom_vol_adj.dropna()

    # --- Vol-adjusted 12–1 momentum: 12–1 return / vol over same window ---
    if momentum_mode == "vol_adj_12_1":
        prev_target_far  = date_curr - pd.DateOffset(months=L+1)
        prev_target_near = date_curr - pd.DateOffset(months=1)

        idx_far  = dates.get_indexer([prev_target_far],  method="ffill")[0]
        idx_near = dates.get_indexer([prev_target_near], method="ffill")[0]

        if idx_far == -1 or idx_near == -1:
            return pd.Series(dtype=float)

        date_far  = dates[idx_far]
        date_near = dates[idx_near]

        close_far  = close_wide.loc[date_far]
        close_near = close_wide.loc[date_near]

        mom_12_1 = (close_near / close_far) - 1.0

        # vol over the same 12–1 window
        window_start = date_far
        window_end   = date_near

        mask = (close_wide.index >= window_start) & (close_wide.index <= window_end)
        cl_slice = close_wide.loc[mask]
        daily_ret = cl_slice.pct_change().dropna(how="all")

        vol = daily_ret.std()
        mom_vol_adj_12_1 = mom_12_1 / vol.replace(0, np.nan)

        return mom_vol_adj_12_1.dropna()

    # fallback
    return simple_mom.dropna()


# =====================================================
# LOAD & PREPARE DATA
# =====================================================

print("Loading data...")
raw = pd.read_csv(csv_path, parse_dates=[date_col], dayfirst=True, low_memory=False)

raw = raw[[date_col, ticker_col, open_col, close_col, volume_col]].rename(
    columns={
        date_col: "Date",
        ticker_col: "Ticker",
        open_col: "Open",
        close_col: "Close",
        volume_col: "Volume"
    }
)

raw = raw.dropna(subset=["Date", "Ticker", "Open", "Close", "Volume"])
raw = raw.sort_values(["Date", "Ticker"])

# =====================================================
# APPLY UNIVERSE FILTERS (static, using full history)
# =====================================================

if use_universe_filters:
    print("Applying universe filters (median price / median dollar volume)...")

    raw["DollarVol"] = raw["Close"] * raw["Volume"]

    grouped = raw.groupby("Ticker")
    median_price = grouped["Close"].median()
    median_dv    = grouped["DollarVol"].median()

    eligible = median_price.index[
        (median_price >= min_median_price) &
        (median_dv    >= min_median_dollar_vol)
    ]

    print(f"Total symbols before filters: {raw['Ticker'].nunique()}")
    print(f"Symbols passing filters:       {len(eligible)}")

    raw = raw[raw["Ticker"].isin(eligible)].copy()
else:
    print("Universe filters disabled. Using all symbols.")

print("Building wide price matrices...")

open_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Open",
    aggfunc="last"
).sort_index()

close_wide = raw.pivot_table(
    index="Date",
    columns="Ticker",
    values="Close",
    aggfunc="last"
).sort_index()

open_wide.index = pd.to_datetime(open_wide.index)
close_wide.index = pd.to_datetime(close_wide.index)

all_dates = open_wide.index.union(close_wide.index)
open_wide  = open_wide.reindex(all_dates).sort_index()
close_wide = close_wide.reindex(all_dates).sort_index()

dates_index_close = close_wide.index
dates_index_open  = open_wide.index

month_starts_candidates = open_wide.resample("MS").first().index
print(f"Number of candidate rebalance months: {len(month_starts_candidates)}")


# =====================================================
# BACKTEST LOOP (FULL HISTORY)
# =====================================================

rebalance_records = []
returns_records   = []

print("Starting backtest...")

for dt in month_starts_candidates:
    for L in lookback_months_list:

        mom = compute_momentum_score(close_wide, dt, L)
        mom = mom.dropna()

        if mom.shape[0] < 2 * min(k_list):
            continue

        ranked = mom.sort_values(ascending=False)

        for k in k_list:
            if ranked.shape[0] < 2 * k:
                continue

            longs  = list(ranked.index[:k])
            shorts = list(ranked.index[-k:])

            idx_entry = dates_index_open.get_indexer([dt], method="backfill")[0]
            if idx_entry == -1:
                continue
            entry_date = dates_index_open[idx_entry]
            entry_row  = open_wide.loc[entry_date]

            for H in forward_horizons:
                exit_target = dt + pd.DateOffset(months=H)
                idx_exit = dates_index_close.get_indexer([exit_target], method="backfill")[0]
                if idx_exit == -1:
                    continue

                exit_date = dates_index_close[idx_exit]
                exit_row  = close_wide.loc[exit_date]

                universe_tickers = list(set(longs) | set(shorts))
                entry_vals = entry_row[universe_tickers]
                exit_vals  = exit_row[universe_tickers]

                rtns = (exit_vals / entry_vals) - 1.0
                rtns = rtns.dropna()
                if rtns.empty:
                    continue

                rtns_net = rtns - (2.0 * tx_cost_pct)

                # Long leg
                valid_longs = [t for t in longs if t in rtns_net.index]
                if len(valid_longs) == 0:
                    continue
                long_rtns = rtns_net.loc[valid_longs]
                w_long = make_weights_for_side(valid_longs, side="long")
                w_long = w_long.loc[long_rtns.index]
                long_ret = float((w_long * long_rtns).sum())

                # Short leg
                valid_shorts = [t for t in shorts if t in rtns_net.index]
                if len(valid_shorts) == 0:
                    continue
                short_rtns_raw = rtns_net.loc[valid_shorts]
                w_short = make_weights_for_side(valid_shorts, side="short")
                w_short = w_short.loc[short_rtns_raw.index]
                short_ret_raw = float((w_short * short_rtns_raw).sum())
                short_ret     = -short_ret_raw

                strat_ret_gross = long_ret + short_ret

                days_held = max((exit_date - entry_date).days, 1)
                holding_cost = financing_rate_annual * (days_held / 365.0)

                strat_ret_net = strat_ret_gross - holding_cost

                returns_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "entry_date": entry_date,
                    "exit_date": exit_date,
                    "days_held": days_held,
                    "financing_cost": holding_cost,
                    "long_return": float(long_ret),
                    "short_return": float(short_ret),
                    "strat_return_gross": float(strat_ret_gross),
                    "strat_return": float(strat_ret_net)
                })

                rebalance_records.append({
                    "rebalance_date": dt,
                    "lookback_months": L,
                    "k": k,
                    "horizon_months": H,
                    "longs": ",".join(longs),
                    "shorts": ",".join(shorts),
                    "entry_date": entry_date,
                    "exit_date": exit_date
                })

print("Backtest loop completed.")

reb_df = pd.DataFrame(rebalance_records)
ret_df = pd.DataFrame(returns_records)

reb_df.to_csv(rebalance_csv_out, index=False)
ret_df.to_csv(returns_csv_out, index=False)

print(f"Rebalance book saved to: {rebalance_csv_out}")
print(f"Returns table saved to:  {returns_csv_out}")


# =====================================================
# EVALUATION & OPTIMIZATION (2019 ONWARDS)
# =====================================================

ret_df["rebalance_date"] = pd.to_datetime(ret_df["rebalance_date"])
ret_eval = ret_df[ret_df["rebalance_date"] >= evaluation_start].copy()

if ret_eval.empty:
    print(f"No rebalance observations on or after {evaluation_start.date()}. "
          "Cannot perform 2019-onwards optimization.")
else:
    combos_2019 = (
        ret_eval.groupby(["lookback_months", "k", "horizon_months"])
        .size()
        .reset_index(name="n_trades")
    )
    print("\nParameter combinations with trades in 2019+ window:")
    print(combos_2019.to_string(index=False))

    perf = (
        ret_eval
        .groupby(["lookback_months", "k", "horizon_months"])["strat_return"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={
            "mean":  "mean_forward_return",
            "std":   "std_forward_return",
            "count": "n_obs"
        })
    )

    perf["mean_per_month"] = perf["mean_forward_return"] / perf["horizon_months"]
    perf["std_per_month"]  = perf["std_forward_return"] / np.sqrt(perf["horizon_months"])

    perf["ann_return"] = (1.0 + perf["mean_per_month"])**12 - 1.0
    perf["ann_vol"]    = perf["std_per_month"] * np.sqrt(12.0)
    perf["sharpe_ann"] = np.where(
        perf["ann_vol"] > 0,
        perf["ann_return"] / perf["ann_vol"],
        np.nan
    )

    perf.to_csv(summary_csv_out, index=False)
    print(f"\nParameter summary (2019 onwards) saved to: {summary_csv_out}")

    perf_valid = perf[perf["n_obs"] >= min_obs_for_config].copy()
    perf_valid_H1 = perf_valid[perf_valid["horizon_months"] == 1].copy()
    perf_to_use = perf_valid_H1 if not perf_valid_H1.empty else perf_valid

    if perf_to_use.empty:
        print("No configurations meet the minimum observation requirement (2019+). "
              "Stopping before QuantStats report.")
    else:
        best_row = perf_to_use.sort_values("sharpe_ann", ascending=False).iloc[0]

        best_L = int(best_row["lookback_months"])
        best_k = int(best_row["k"])
        best_H = int(best_row["horizon_months"])

        best_sharpe = best_row["sharpe_ann"]
        best_annret = best_row["ann_return"]

        if best_sharpe > 1.0:
            quality = "✅ GOOD (Sharpe > 1.0)"
        elif best_sharpe > 0.0:
            quality = "🟡 OK (Sharpe > 0)"
        else:
            quality = "❌ BAD (Sharpe ≤ 0)"

        print("\n=== BEST CONFIGURATION SUMMARY (2019 onwards) ===")
        print(f"Experiment:             {exp_tag}")
        print(f"Momentum mode:          {momentum_mode}")
        print(f"Lookback (months):      {best_L}")
        print(f"Number of longs/shorts: {best_k}")
        print(f"Holding horizon (M):    {best_H}")
        print(f"Annualized return:      {best_annret:.2%}")
        print(f"Annualized vol:         {best_row['ann_vol']:.2%}")
        print(f"Annualized Sharpe:      {best_sharpe:.2f}")
        print(f"Number of rebalances:   {int(best_row['n_obs'])}")
        print(f"Iteration quality:      {quality}")

        # =====================================================
        # BUILD SERIES FOR QUANTSTATS (DAILYIZED)
        # AND DAILY EQUITY + EXPOSURE
        # =====================================================

        best_returns_eval = (
            ret_eval[
                (ret_eval["lookback_months"] == best_L) &
                (ret_eval["k"] == best_k) &
                (ret_eval["horizon_months"] == best_H)
            ]
            .copy()
            .sort_values("entry_date")
        )

        best_returns_eval["entry_date"] = pd.to_datetime(best_returns_eval["entry_date"])
        best_returns_eval["exit_date"]  = pd.to_datetime(best_returns_eval["exit_date"])

        daily_ret = pd.Series(dtype=float)

        for _, row in best_returns_eval.iterrows():
            start = row["entry_date"]
            end   = row["exit_date"]
            R     = row["strat_return"]

            days = pd.date_range(start, end, freq="B")
            if len(days) == 0:
                continue

            r_daily = (1.0 + R)**(1.0 / len(days)) - 1.0

            for d in days:
                if d in daily_ret.index:
                    daily_ret.loc[d] = (1.0 + daily_ret.loc[d]) * (1.0 + r_daily) - 1.0
                else:
                    daily_ret.loc[d] = r_daily

        daily_ret = daily_ret.sort_index()
        daily_ret.name = f"{exp_tag}_L{best_L}_k{best_k}_H{best_H}"

        if not daily_ret.empty:
            full_idx = pd.bdate_range(daily_ret.index.min(), daily_ret.index.max())
            daily_ret = daily_ret.reindex(full_idx).fillna(0.0)

        daily_equity = (1.0 + daily_ret).cumprod() * starting_equity
        daily_gross_exposure = daily_equity * gross_leverage

        exposure_df = pd.DataFrame({
            "equity": daily_equity,
            "gross_exposure": daily_gross_exposure,
            "exposure_pct_of_initial": daily_gross_exposure / starting_equity
        })
        exposure_df.index.name = "date"
        exposure_df.to_csv(exposure_csv_out)

        bench_daily = pd.Series(0.0, index=daily_ret.index)
        bench_daily.name = "Zero_Benchmark"

        os.makedirs(os.path.dirname(os.path.abspath(qs_html_out)), exist_ok=True)

        print("\nGenerating QuantStats HTML report...")

        qs.reports.html(
            daily_ret,
            benchmark=bench_daily,
            title=f"{exp_tag} – Best Config (mode={momentum_mode})",
            output=qs_html_out
        )

        print(f"QuantStats HTML report saved to: {os.path.abspath(qs_html_out)}")
        print(f"Daily equity & exposure saved to: {os.path.abspath(exposure_csv_out)}")

print("\nFinished.")



===== RUNNING exp4_liquidity_vol_adj_12_1_momentum =====

Loading data...
Applying universe filters (median price / median dollar volume)...
Total symbols before filters: 2030
Symbols passing filters:       1708
Building wide price matrices...
Number of candidate rebalance months: 116
Starting backtest...


C:\Users\avina\AppData\Local\Temp\ipykernel_3136\1990814360.py:240: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = cl_slice.pct_change().dropna(how="all")
C:\Users\avina\AppData\Local\Temp\ipykernel_3136\1990814360.py:240: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = cl_slice.pct_change().dropna(how="all")
C:\Users\avina\AppData\Local\Temp\ipykernel_3136\1990814360.py:240: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or 

Backtest loop completed.
Rebalance book saved to: exp4_liquidity_vol_adj_12_1_momentum_rebalances.csv
Returns table saved to:  exp4_liquidity_vol_adj_12_1_momentum_returns.csv

Parameter combinations with trades in 2019+ window:
 lookback_months  k  horizon_months  n_trades
               3  5               1        75
               3 10               1        76
               3 20               1        76
               3 50               1        74
               6  5               1        75
               6 10               1        76
               6 20               1        76
               6 50               1        75
              12  5               1        74
              12 10               1        74
              12 20               1        76
              12 50               1        77
              24  5               1        74
              24 10               1        75
              24 20               1        76
              24 50               1

C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\quantstats\stats.py:497: UserWarning: No non-zero returns found for win rate calculation, returning 0.0
  warn("No non-zero returns found for win rate calculation, returning 0.0")
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\avina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

QuantStats HTML report saved to: D:\work\Trade Analysis\exp4_liquidity_vol_adj_12_1_momentum_QS_best_2019_onwards.html
Daily equity & exposure saved to: D:\work\Trade Analysis\exp4_liquidity_vol_adj_12_1_momentum_daily_equity_exposure.csv

Finished.
